## UNION vs UNION ALL

- spojuje tabulky vertikálně
- UNION =\> vyřazuje duplicitní řádky
- UNION ALL =\> spojí oba dotazy, aniž by duplicity vyřadil

Ukázka 1

In [ ]:
SELECT  w.name,
        SUM(t.nkill) AS mrtvi,
        SUM(IIF(t.nhostkid = -99,NULL,nhostkid)) AS rukojmi, 
        SUM(t.nwound) AS raneni 
FROM czechitas.teror2 t
JOIN czechitas.weaptype1 w
ON t.weaptype1=w.id
GROUP BY w.name
UNION ALL
SELECT  w.name,
        SUM(nkill) AS mrtvi,
        SUM(IIF(t.nhostkid = -99,NULL,nhostkid)) AS rukojmi, 
        SUM(t.nwound) AS raneni 
FROM czechitas.teror2 t
JOIN czechitas.weaptype2 w
ON t.weaptype2=w.id
GROUP BY w.name
;

Ukázka 2

In [ ]:
SELECT w.name 
FROM czechitas.teror2 t
JOIN czechitas.weaptype1 w
ON t.weaptype1=w.id
UNION ALL 
SELECT w.name
FROM czechitas.teror2 t
JOIN czechitas.weaptype2 w
ON t.weaptype2=w.id
;

UKázka 3

In [ ]:
SELECT w.name 
FROM czechitas.teror2 t
JOIN czechitas.weaptype1 w
ON t.weaptype1=w.id
UNION ---- pozor odstraní duplicity 
SELECT w.name
FROM czechitas.teror2 t
JOIN czechitas.weaptype2 w
ON t.weaptype2=w.id;

## WINDOW FUNKCE

- OKNO = vytvoříme další sloupec a do řádku se doplní hodnota společná pro nějakou skupinu (př. zemi)
- lze tim zajistit kumulativní součet, přiřazené pořadí, číslo řádku a další...

Vybere seřazené organizace podle počtu obětí sestupně a přiřadí jim pořadí (rank)

In [ ]:
SELECT  gname, 
        SUM(nkill), 
        RANK() OVER (ORDER BY SUM(nkill) DESC) AS rank
FROM teror
WHERE nkill IS NOT NULL
GROUP BY gname
ORDER BY SUM(nkill) DESC;

Vybere seřazené organizace podle počtu obětí sestupně a přiřadí jim pořadí (rank) v rámci roku.

In [ ]:
SELECT  gname 
        ,iyear 
        ,SUM(nkill)
        ,ROW_NUMBER() OVER (ORDER BY SUM(nkill) DESC) AS rn
        ,RANK() OVER (PARTITION BY iyear ORDER BY SUM(nkill) DESC) AS rank
FROM teror
WHERE nkill IS NOT NULL
GROUP BY gname, iyear
ORDER BY sum(nkill) DESC;

Vybere seřazené organizace podle počtu obětí sestupně a přiřadí jim pořadí (rank) v rámci roku. Nakonec vybereme jen první tři z každého roku.

In [ ]:
SELECT vyber.*
FROM (
        SELECT  gname
                ,iyear
                ,SUM(nkill) AS pocet_mrtvych
                ,ROW_NUMBER() OVER (ORDER BY SUM(nkill) DESC) AS rn
                ,RANK() OVER (PARTITION BY iyear ORDER BY SUM(nkill) DESC) AS rank
        FROM teror
        WHERE nkill IS NOT NULL
        GROUP BY gname, iyear
        ) AS vyber
WHERE rank <= 3;